In [1]:
# coding:utf-8
import sys  
reload(sys)
#解决编码问题
sys.setdefaultencoding('utf8')
import requests
from bs4 import BeautifulSoup

In [2]:
url='https://catalog.mann-filter.cn/CN/chi/vehicle/MANN-FILTER Katalog China'
# 请求腾讯新闻的URL，获取其text文本
mannData = requests.get(url).text
# 对获取到的文本进行解析

In [ ]:
soup = BeautifulSoup(mannData, "lxml") 
for hersteller in soup.find(id="dropdownsHerstellerSelectItems").select(".rf-sel-opt"):
    herstellerName = hersteller.text.replace('/','~'); 
    csvFile = open('./data/'+herstellerName+".csv", "w")
    csvFile.write('生产商,车系,车型,发动机型号,千瓦,马力,制造年份,空气滤清器,机油滤清器,燃油滤清器,空调滤清器,其它滤清器 \n')

    url_her = url+'/车辆/乘用车+轻型货车/'+herstellerName
    herstellerHtml = requests.get(url_her).text 
    herstellerSoup = BeautifulSoup(herstellerHtml, "lxml")

    for modelItem in herstellerSoup.find(id="dropdownsModellreihe_SelectItems").select('.rf-sel-opt'):
        print('生产商'+ herstellerName +' 车系:' + modelItem.getText())
        modelText = modelItem.getText().replace('/','~').split('|')[0];
        url_model = url_her+'/'+modelText
    #     print url_model

        # 车辆型号
        model_html = requests.get(url_model).text 
        model_soup = BeautifulSoup(model_html, "lxml")
        #(".tableContent"):.strip().replace('\n','|') + '\n')
        for modelItem in model_soup.select(".clickable_frame"):
    #         print "  车型：" + ' '.join(modelItem.text.split())

            if modelItem.text.strip() :
                #f.write("  车型：" + ' '.join(_table.text.split())+'\n') 
                type_url = 'https://catalog.mann-filter.cn'+modelItem.a['href']

                index=0
                modelDetil=''
                for modelInfo in modelItem.select(".tableContent"):
                   modelInfoText = ' '.join(modelInfo.text.split())
                   modelDetil += ','+ modelInfoText
    #                if index==0 :
    #                     print '车型'+ modelInfoText
    #                elif index==1:  
    #                     print '发动机型号'+modelInfoText
    #                elif index==2:  
    #                     print '千瓦'+modelInfoText
    #                elif index==3:  
    #                     print '马力'+modelInfoText
    #                elif index==4:  
    #                     print '制造年份'+modelInfoText
    #                else:
    #                     print '---'
    #                index+=1
                filter_url = 'https://catalog.mann-filter.cn'+modelItem.a['href']
               # print type_url
                filter_html  = requests.get(filter_url).text 
                filter_soup = BeautifulSoup(filter_html, "lxml")
                filterDetil=''
                for filterItem in filter_soup.select('.partsText'): 
                    for filterInfo in filterItem.find_all('a'): 
    #                 for filterInfo in filterItem.find_all("div", attrs={"class": "partsInlineText partsText"}): 
                       filterDetil += ','+ ' '.join(filterInfo.text.split())
                print  herstellerName+','+modelText+modelDetil+filterDetil
                csvFile.write(herstellerName+','+modelText+modelDetil+filterDetil+'\n')

    csvFile.close()

# print(_soup1.prettify())